# CNP_func_validation_age

In [3]:
import pandas as pd
import numpy as np
import nibabel as nib
from sklearn.svm import LinearSVR
# from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import VarianceThreshold, SelectKBest, f_regression
import os
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_absolute_error, r2_score
from nilearn.input_data import NiftiMasker
import random
import matplotlib.pyplot as plt
import datetime

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [4]:
# Config
toy = False
run_all = False



#AWS cloud
# path = '/home/ubuntu/fmriprep/'
# output_dir = '/output/' 

#For Daniel's computer
input_dir = 'data/CNP_func/'
output_dir = 'data/CNP_T1/' 


n_jobs = 20 #amount of cores
cv=4

parcellation = '444'

description = 'CNP_func_validation_age_'
log_file = description+datetime.datetime.now().strftime("%y-%m-%d-%H-%M-%S")

In [16]:
# Load data
input_dir = '/Users/danielmlow/Dropbox/neurohack/brain_age/data/CNP_func/'
loaded = np.load(input_dir+'CNP_func_gridsearch_age_18-08-09-22-05-36CNP_func444_train_set.npz')
X_train, y_train, random_subj_train = loaded['a'], loaded['b'], loaded['c']

input_dir = '/Users/danielmlow/Dropbox/neurohack/brain_age/data/NKI_func/'
loaded = np.load(input_dir+'NKI_func_gridsearch_age_18-08-10-19-07-24NKI_func444_train_set.npz')
X_train_NKI, y_train_NKI, random_subj_train_NKI = loaded['a'], loaded['b'], loaded['c']

print(X_train_NKI.shape)
print(X_train.shape)
X_train_NKI = np.reshape(X_train_NKI, (84, 98346))
X_train = np.reshape(X_train, (84, 98346))
X_train.shape

(84, 1, 98346)
(84, 1, 98346)


(84, 98346)

In [39]:
print(y_train.shape)
print(X_train.shape)
X_train = np.reshape(X_train, (84, X_train.shape[2]))

(84,)
(84, 1, 98346)


In [40]:
# X_train, X_validation, y_train, y_validation = train_test_split(X_train, y_train, test_size=0.30, random_state=42)

## Validation 

Grid search returned the best parameters:

* ANOVA k selection: 'all'
* C parameter of SVR: 1


In [41]:
X_train.shape

(58, 98346)

In [22]:
from sklearn.pipeline import Pipeline

if toy:
    X_train = X_train[:8]
    y_train = y_train[:8]
    X_validation = X_validation[:8]
    y_validation = y_validation[:8]


# Remove features with too low between-subject variance
# Here we use a classical univariate feature selection based on F-test,
# namely Anova.
# We have our predictor (SVR), our feature selection (SelectKBest), and now,
# we can plug them together in a *pipeline* that performs the two operations
# successively:
               
anova_svr = Pipeline([
            ('anova', SelectKBest(f_regression, k='all')),
            ('svr', LinearSVR(C=1.))])

# parameters = [{'anova__k': [2000],
#                'svr__C': [0.1,1]}]


# anova_svr.fit(X_train, y_train)
# predictions = anova_svr.predict(X_train_NKI)

# score = mean_absolute_error(y_train_NKI, predictions)
# r2 = r2_score(y_train_NKI, predictions)



# =======
anova_svr.fit(X_train_NKI, y_train_NKI)
predictions = anova_svr.predict(X_train)

score = mean_absolute_error(y_train, predictions)
r2 = r2_score(y_train, predictions)





In [23]:
print(score)
print(r2)
print(predictions)
print(y_train_NKI)

9.602909911288034
-1.072453263378513
[18.15949995 25.07796494 25.41099701 26.71632344 27.49406321 25.18506327
 34.17460763 19.47976179 25.60742506 31.19455168 24.08271234 11.37024104
 21.26987546 16.88477018 26.84553845 23.85160133 31.0520572  13.23991424
 27.49507415 31.61648135 31.4260102  18.41116334 13.89424139 27.17351961
 26.07366819 25.8943376  19.85674136 26.95980391 18.54842692 33.05655099
 16.8500166  34.01535418 33.8425835  14.81359688 16.71590723 21.04135276
 18.32398083 35.94831468 24.1530108  15.85339597 24.56331227 35.56175048
 32.02003304 16.23969985 14.20854712 32.20229463 31.82596375 17.97581115
 24.20918718 24.26765863 28.2619019  18.05448914 24.06668956 15.48025428
 21.99080739 35.01547017 24.59500034 19.78662241 22.91559101 20.83674428
 24.38226132 19.25588164 18.26724058 30.84896311 20.01331805 23.7884209
 23.40639516 25.98307914 23.4628912  20.69277975 22.14535265 32.0638656
 24.99883092 27.68392423 26.58772885 40.11917018 20.60294825 17.40688652
 28.28375755 11.

In [46]:
# print('best score: ',grid.best_score_)
# print('best params: ',grid.best_params_)
# print('\n\nfullresults: \n', grid.cv_results_)
# df1 = pd.DataFrame(grid.cv_results_)
# df1

In [47]:


with open(output_dir+'log_'+log_file+'.txt', '+a') as f:
    f.write('mean absolute error: '+str(score.round(4))+'\n')
    f.write('r2: '+str(r2.round(4))+'\n')
    f.write(str(predictions)+'\n')
#     f.write('\n\nfullresults: \n'+str(grid.cv_results_)+'\n')
#     f.write(str(df1)+'\n')
#     f.write(str(age_pred)+'\n')
#     f.write(str(y_train)+'\n')
#     f.write(str(mean_abs_error)+'\n')
    

In [48]:
r2

-0.27706915864013837

In [ ]:
# age_pred = grid.predict(gm_maps_masked)
# np.mean(np.abs(age_pred-y_age))
# y_age

In [ ]:
# r2 score